In [1]:
!pip install nltk

In [2]:
!python -m pip install --upgrade pip


In [3]:
!pip install flask

In [4]:
!pip install flask fuzzywuzzy pandas

In [5]:
!pip install flask pandas nltk fuzzywuzzy openpyxl

In [6]:
!pip install pandas nltk fuzzywuzzy Flask

In [7]:
!pip install fastapi uvicorn nest-asyncio pandas nltk fuzzywuzzy

In [8]:
!pip install python-multipart

In [9]:
!pip install fastapi uvicorn nest-asyncio pandas nltk fuzzywuzzy

In [10]:
!python -m pip install --upgrade pip setuptools wheel

In [11]:
!pip install spacy --no-deps

In [12]:
!python -m spacy download en_core_web_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz, process
from flask import Flask, request, render_template_string
import re

# Initialize the Flask app
app = Flask(__name__)

# Load the data from Excel
file_path = r"C:\Users\Saleel01\Downloads\Leeds-20241110T220642Z-001\Leeds\Work Done\New folder\datasets\Datasetfinal12_11.xlsx"
excel_data = pd.ExcelFile(file_path)

# Load each sheet into a DataFrame
disease_bodyfn_df = excel_data.parse('30-10-DISEASE VS BODYFN')
bodyfn_assistive_df = excel_data.parse('31-10 BODYFN vs assistive techn')
bodyfn_home_icf_df = excel_data.parse('31-10 BODYFN vs HOME ICF')

# Preprocessing helper function
def preprocess_text(text):
    """Normalize the text: remove special characters, extra spaces, and convert to lowercase."""
    if text is None:
        return ""
    return re.sub(r'[^a-zA-Z0-9\s]', '', text.strip().lower())

# Normalize the dataset for fuzzy matching
disease_bodyfn_df['Disease'] = disease_bodyfn_df['Disease'].apply(preprocess_text)
bodyfn_assistive_df['Symptoms'] = bodyfn_assistive_df['Symptoms'].apply(preprocess_text)
bodyfn_home_icf_df['Symptoms'] = bodyfn_home_icf_df['Symptoms'].apply(preprocess_text)

# Helper function to find the closest match with fuzzy matching
def find_closest_match(input_text, text_list, threshold=80):
    """Find the closest match for the input text within the text list, using fuzzy matching."""
    # First check for exact match
    if input_text in text_list:
        return input_text

    # Fallback to fuzzy match
    match = process.extractOne(input_text, text_list, scorer=fuzz.partial_ratio)
    return match[0] if match and match[1] >= threshold else None

# Function to retrieve details based on disease input
def get_disease_details(disease_name):
    disease_name = preprocess_text(disease_name)
    disease_list = disease_bodyfn_df['Disease'].tolist()
    matched_disease = find_closest_match(disease_name, disease_list)
    if matched_disease is None:
        return None, [], [], False  # Add a boolean to indicate if it's a disease

    disease_info = disease_bodyfn_df[disease_bodyfn_df['Disease'] == matched_disease]
    symptoms = disease_info['Symptoms'].values[0].split(', ') if not disease_info.empty else []
    assistive_mapping, home_function_mapping = [], []

    all_symptoms_assistive = bodyfn_assistive_df['Symptoms'].tolist()
    all_symptoms_home = bodyfn_home_icf_df['Symptoms'].tolist()

    for symptom in symptoms:
        symptom = preprocess_text(symptom)
        assistive_tech = "No assistive technology found"
        home_functions = "No home adaptations found"

        # Match symptoms to assistive technology from Sheet 2
        matched_symptom_assistive = find_closest_match(symptom, all_symptoms_assistive, threshold=80)
        if matched_symptom_assistive:
            tech_info = bodyfn_assistive_df[bodyfn_assistive_df['Symptoms'] == matched_symptom_assistive]
            assistive_tech = tech_info['Assistive Technology'].values[0] if not tech_info.empty else assistive_tech

        # Match symptoms to home functions from Sheet 3
        matched_symptom_home = find_closest_match(symptom, all_symptoms_home, threshold=80)
        if matched_symptom_home:
            home_info = bodyfn_home_icf_df[bodyfn_home_icf_df['Symptoms'] == matched_symptom_home]
            home_functions = ", ".join(home_info['Home functions'].dropna().tolist()) or home_functions

        assistive_mapping.append((symptom, assistive_tech))
        home_function_mapping.append((symptom, home_functions))

    return matched_disease, assistive_mapping, home_function_mapping, symptoms, True  # Return symptoms as well

# Function to check if the input matches a symptom from the list
def check_if_symptom(input_text):
    """Check if the input text matches any symptom in the symptoms list."""
    symptoms_list = bodyfn_assistive_df['Symptoms'].tolist() + bodyfn_home_icf_df['Symptoms'].tolist()
    input_text = preprocess_text(input_text)
    matched_symptom = find_closest_match(input_text, symptoms_list)
    return matched_symptom

# HTML Templates (with requested changes)
home_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>HOMEAI-ENABLE</title>
    <style>
        body {
            font-family: 'Arial', sans-serif;
            background-color: #d4f7d4; /* Light Green Background */
            color: #333;
            margin: 0;
            padding: 0;
        }
        .navbar {
            background: #0056b3;
            padding: 15px 0;
            display: flex;
            justify-content: center;
            align-items: center;
            text-align: center;
        }
        .navbar a {
            color: white;
            text-decoration: none;
            font-weight: bold;
            margin: 0 15px;
            font-size: 18px;
        }
        .navbar a:hover {
            text-decoration: underline;
            color: #ffdd57;
        }
        .container {
            max-width: 800px;
            margin: 50px auto;
            text-align: center;
            color: #333;
        }
        h1 {
            color: #0056b3;
        }
        p {
            font-size: 20px;
        }
        ul {
            text-align: left;
            font-size: 18px;
        }
    </style>
</head>
<body>
    <div class="navbar">
        <a href="/">Home</a>
        <a href="/disease">Enter Disease Details</a>
        <a href="/contact">Contact Us</a>
    </div>
    <div class="container">
        <h1>Welcome to HOMEAI-ENABLE</h1>
        <h2>Empowering lives through smart assistive solutions</h2>
        <p>At HOMEAI-ENABLE, we make your home a haven of comfort, safety, and independence. Whether you or a loved one are living with a specific condition or seeking to enhance accessibility, our platform connects you to the latest assistive technologies and customized home adaptations.</p>
        <ul>
            <li><strong>Search for assistive technologies:</strong> Discover innovative tools designed to support various diseases and symptoms.</li>
            <li><strong>Explore home adaptations:</strong> Find tailored solutions to create a safe, accessible, and comfortable living environment.</li>
            <li><strong>Personalized guidance:</strong> Get expert recommendations for integrating technology and design that meet your unique needs.</li>
        </ul>
        <p>Start your journey to a smarter, more inclusive home today.</p>
    </div>
</body>
</html>
"""

disease_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>HOMEAI-ENABLE</title>
    <style>
        body {
            font-family: 'Arial', sans-serif;
            background-color: #d4f7d4; /* Light Green Background */
            color: #333;
            margin: 0;
            padding: 0;
        }
        .navbar {
            background: #0056b3;
            padding: 15px 0;
            display: flex;
            justify-content: center;
            align-items: center;
            text-align: center;
        }
        .navbar a {
            color: white;
            text-decoration: none;
            font-weight: bold;
            margin: 0 15px;
            font-size: 18px;
        }
        .navbar a:hover {
            text-decoration: underline;
            color: #ffdd57;
        }
        .container {
            max-width: 800px;
            margin: 30px auto;
            background: #ffffff;
            border-radius: 15px;
            box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.1);
            padding: 20px;
        }
        h1 {
            text-align: center;
            color: #0056b3;
        }
        input[type="text"] {
            width: 100%;
            padding: 25px;
            font-size: 22px; /* Increased font size for better readability */
            margin: 15px 0;
            border: 2px solid #ddd;
            border-radius: 5px;
        }
        button {
            padding: 15px 30px;
            font-size: 22px;
            background-color: #0056b3;
            color: white;
            border: none;
            border-radius: 5px;
            cursor: pointer;
        }
        button:hover {
            background-color: #003d75;
        }
        .output {
            font-size: 22px;
            color: #333;
            margin: 20px 0;
        }
        .output ul {
            list-style-type: none;
            padding-left: 0;
        }
        .output li {
            margin-bottom: 10px;
        }
    </style>
</head>
<body>
    <div class="navbar">
        <a href="/">Home</a>
        <a href="/disease">Enter Disease Details</a>
        <a href="/contact">Contact Us</a>
    </div>
    <div class="container">
        <h1>Enter Disease or Symptom</h1>
        <form method="POST">
            <input type="text" name="disease_name" placeholder="Enter disease or symptom (e.g., Stroke, Cough)" required>
            <button type="submit">Search</button>
        </form>
        {% if message %}
            <p class="output">{{ message }}</p>
        {% endif %}
        {% if disease %}
            <p class="output">
                Symptoms: <strong>{{ symptoms|join(', ') }}</strong>
            </p>
            <ul class="output">
                {% for symptom, tech in assistive_mapping %}
                    <li>For <strong>{{ symptom }}</strong>, consider using <em>{{ tech }}</em>.</li>
                {% endfor %}
            </ul>
            <ul class="output">
                {% for symptom, adaptation in home_function_mapping %}
                    <li>For <strong>{{ symptom }}</strong>, consider: <em>{{ adaptation }}</em>.</li>
                {% endfor %}
            </ul>
        {% endif %}
    </div>
</body>
</html>
"""

@app.route('/')
def home():
    return render_template_string(home_template)

@app.route('/disease', methods=['GET', 'POST'])
def disease():
    disease_name = None
    assistive_mapping = []
    home_function_mapping = []
    message = None
    symptoms = []
    is_disease = False

    if request.method == 'POST':
        disease_name = request.form['disease_name']
        matched_symptom = check_if_symptom(disease_name)

        if matched_symptom:
            assistive_tech = "No assistive technology found"
            home_functions = "No home adaptations found"

            matched_symptom_assistive = find_closest_match(matched_symptom, bodyfn_assistive_df['Symptoms'].tolist())
            if matched_symptom_assistive:
                assistive_tech = bodyfn_assistive_df[bodyfn_assistive_df['Symptoms'] == matched_symptom_assistive]['Assistive Technology'].values[0]

            matched_symptom_home = find_closest_match(matched_symptom, bodyfn_home_icf_df['Symptoms'].tolist())
            if matched_symptom_home:
                home_functions = ", ".join(bodyfn_home_icf_df[bodyfn_home_icf_df['Symptoms'] == matched_symptom_home]['Home functions'].dropna().tolist())

            assistive_mapping.append((matched_symptom, assistive_tech))
            home_function_mapping.append((matched_symptom, home_functions))

            message = f"Got it! {matched_symptom.capitalize()} is a symptom. You can consider the following assistive technologies and home adaptations for better management."

        else:
            disease_info, assistive_mapping, home_function_mapping, symptoms, is_disease = get_disease_details(disease_name)

            if disease_info is None:
                message = "No disease found. Please enter a valid disease name or symptom."
            else:
                message = "Here are some assistive technologies and home adaptations for this disease."

    return render_template_string(disease_template, disease=disease_name, assistive_mapping=assistive_mapping,
                                  home_function_mapping=home_function_mapping, message=message, symptoms=symptoms, is_disease=is_disease)

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False, port=5015)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5015 (Press CTRL+C to quit)
127.0.0.1 - - [17/Dec/2024 13:02:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2024 13:02:39] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [17/Dec/2024 13:02:42] "GET /disease HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2024 13:02:45] "POST /disease HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2024 13:02:47] "GET /disease HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2024 13:02:57] "POST /disease HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2024 13:10:01] "POST /disease HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2024 13:11:05] "POST /disease HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2024 13:13:21] "POST /disease HTTP/1.1" 200 -
